In [ ]:
%%capture
!pip install pytorch-lightning
!pip install torchinfo
!pip install wandb
from torchinfo import summary


from google.colab import drive
import torch
import torchvision
import torchvision.transforms as transforms
import torchvision.transforms.functional as TF
import torchvision.models as models
import torchmetrics
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.tensorboard import SummaryWriter
import wandb
from pytorch_lightning.loggers import WandbLogger, TensorBoardLogger
from torch.utils.data import DataLoader, Dataset
from pytorch_lightning.callbacks import ModelCheckpoint
from torchvision.io import read_image
import pytorch_lightning as pl
import matplotlib.pyplot as plt
from PIL import Image
import io
import os
from os import listdir
from os.path import isfile, join

drive.mount('/content/drive')

In [ ]:
class PneumoniaClassifier(pl.LightningModule):

    def __init__(self, learning_rate):
        super().__init__()

        import wandb
        from pytorch_lightning.loggers import WandbLogger
        wandb.init()

        densenet = torchvision.models.densenet121(pretrained=True) 
        self.model = densenet
        
        self.preds = []
        self.truth = []

        

        output_feats = self.model.classifier.out_features
        classifier = nn.Sequential(*[nn.Linear(output_feats, 2)])
        self.classifier = classifier
        
        self.learning_rate = learning_rate
        
        self.valid_acc = torchmetrics.Accuracy()
        self.test_acc = torchmetrics.Accuracy()
        self.confusion_matrix = torchmetrics.ConfusionMatrix(num_classes=2, average='macro')

        self.val_f1 = torchmetrics.F1Score(num_classes=2, average='macro')
        self.val_auroc = torchmetrics.AUROC(num_classes=2, average='macro')

        self.test_f1 = torchmetrics.F1Score(num_classes=2, average='macro')
        self.test_auroc = torchmetrics.AUROC(num_classes=2, average='macro')

        self.current_f1 = 0
        self.max_f1 = 0

    def forward(self, x):

        x = self.model(x)       
        x = torch.squeeze(x)
        x = self.classifier(x)

        return x

    def cross_entropy_loss(self, logits, labels):
        #return F.binary_cross_entropy(logits,labels)
        return F.cross_entropy(logits, labels)
        # return F.cross_entropy(logits, labels, weight=torch.cuda.FloatTensor([100,1]))

    def training_step(self, train_batch, batch_idx):
        x, y = train_batch
        logits = self.forward(x)
        loss = self.cross_entropy_loss(logits, y)
        self.log('train_loss', loss)
        columns = ["train_loss",'learningrate']
        my_table = wandb.Table(columns=columns)
        my_table.add_data(loss,self.learning_rate)

        wandb.log({"custom_data_table": my_table})
        return loss

    def validation_step(self, val_batch, batch_idx):
        x, y = val_batch
        logits = self.forward(x)
        loss = self.cross_entropy_loss(logits, y)
        self.log('val_loss', loss)
        self.valid_acc(logits, y)
        self.log('valid_acc', self.valid_acc, prog_bar = True)
        self.val_f1(logits, y)
        self.log("valid_f1", self.val_f1)
        self.val_auroc(logits,y)
        self.log('valid_auroc', self.val_auroc)
        self.confusion_matrix.update(logits, y)
        
    def on_validation_epoch_end(self):
        print("Confusion matrix")
        print(self.confusion_matrix.compute())
        self.confusion_matrix.reset()

    def test_step(self, test_batch, batch_idx):
        x, y = test_batch
        logits = self.forward(x)
        loss = self.cross_entropy_loss(logits, y)
        self.log('test_loss', loss)
        self.test_acc(logits, y)
        self.log('test_acc', self.test_acc)
        self.test_f1.update(logits, y)
        self.log('test_f1', self.test_f1)
        self.test_auroc(logits,y)
        self.log('test_auroc', self.test_auroc)
        self.confusion_matrix.update(logits, y)
        self.truth.append(y)
        self.preds.append(logits)

          #self.log("images_testset",show_img)
        predz = torch.argmax(logits, -1)
      # Log the images as wandb Image
        images_to_log = [wandb.Image(x, caption=f"Pred:{predz}, Label:{y}") 
                          for x, predz, y in zip(x[:32], #nr_samples = 32 batchsize
                                                predz[:32], 
                                                y[:32])]

        #val_table = wandb.Table(data=images_to_log)
        #wandb.log({'images_table':val_table})  

        wandb.log({'images_test_prediction':images_to_log})  
        #        trainer.logger.experiment.log({
        
        columns = ["loss",'accuracy','f1','AUROC']
        my_table_test = wandb.Table(columns=columns)
        my_table_test.add_data(loss,self.test_acc,self.test_f1,self.test_auroc)

        wandb.log({"custom_data_table": my_table_test})



    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=self.learning_rate)
        # optimizer = torch.optim.Adam(self.parameters(), lr=self.learning_rate, weight_decay=0.01)
        return optimizer

In [ ]:
from torchvision.transforms.transforms import CenterCrop
from torch.autograd._functions.tensor import Resize
from torchvision.datasets import ImageFolder

class PneumoniaDataModule(pl.LightningDataModule):
    def setup(self, stage):

        data_path = "/content/drive/MyDrive/chest_xray"
        transform = transforms.Compose([
             transforms.Resize((224,224)),
             transforms.Grayscale(3),
             transforms.ToTensor(),
             #transforms.RandomAffine(degrees=12, scale=(.9, 1.1)),
             #transforms.RandomCrop((224,224)),
             transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))])
        
        test_transform = transform = transforms.Compose([
             transforms.Resize((224,224)),
             transforms.Grayscale(3),
             transforms.ToTensor(),
             transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))])
      
        #aug_trainset = torchvision.datasets.DatasetFolder(data_path+'/train/NORMAL', transform=transform)
        trainset = torchvision.datasets.ImageFolder(data_path+'/train', transform=transform)
        valset = torchvision.datasets.ImageFolder(data_path+'/val', transform=transform)
        testset = torchvision.datasets.ImageFolder(data_path+'/test',transform=test_transform)
        print(len(trainset)/(len(trainset) + len(testset) + len(valset)))
        print(len(testset)/(len(trainset) + len(testset) + len(valset)))
        print((len(trainset) + len(testset) + len(valset)))


        train_size = int(0.9 * len(trainset))
        test_size = len(trainset) - train_size
        trainset, additional_val_dataset = torch.utils.data.random_split(trainset, [train_size, test_size])
        valset = torch.utils.data.ConcatDataset([valset, additional_val_dataset])

        self.trainset = trainset
        #print("tr_set", len(trainset))
        self.valset = valset
        self.testset = testset
    
    def train_dataloader(self):
        return DataLoader(self.trainset, batch_size=32,
                                          shuffle=True, num_workers=2)

    def val_dataloader(self):
        return DataLoader(self.valset, batch_size=32,
                                         shuffle=False, num_workers=2)
        
    def test_dataloader(self):
        return DataLoader(self.testset, batch_size=32,
                                         shuffle=False, num_workers=2)

In [ ]:
checkpoint_callback = ModelCheckpoint(monitor="valid_f1", filename="sample-pneumonia-{epoch:02d}-{valid_f1:.2f}", mode="max", save_top_k=5)

torch.cuda.empty_cache()
AVAIL_GPUS = min(1, torch.cuda.device_count())

model = PneumoniaClassifier(learning_rate=1e-5)

data_module = PneumoniaDataModule()

wandb_logger = WandbLogger(save_dir='wdb_log',name='densenet_model_no_weight',project="DENSENET", log_model="all")
tb_logger = TensorBoardLogger(save_dir="tb_logs", name="densenet_model_no_weight")

wandb.watch(model, log='all')



#print(summary(model.to("cuda"),input_size=(32,3,224,224)))


/usr/local/lib/python3.7/dist-packages/torchmetrics/utilities/prints.py:36: UserWarning: Metric `AUROC` will save all targets and predictions in buffer. For large datasets this may lead to large memory footprint.
  warnings.warn(*args, **kwargs)
/usr/local/lib/python3.7/dist-packages/pytorch_lightning/loggers/wandb.py:348: UserWarning: There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse this run. If this is not desired, call `wandb.finish()` before instantiating `WandbLogger`.
  "There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse"


[]

In [ ]:
from torchsummary import summary
print(summary(model,(3, 224, 224)))

RuntimeError: ignored

In [ ]:
trainer = pl.Trainer(gpus = AVAIL_GPUS, auto_lr_find=True, max_epochs = 10, callbacks = [checkpoint_callback],logger=[wandb_logger,tb_logger], track_grad_norm='inf')
#trainer = pl.Trainer(gpus = AVAIL_GPUS, auto_lr_find=Trutrainer = pl.Trainer(gpus = AVAIL_GPUS, auto_lr_find=True, max_epochs = 11)

trainer.fit(model, data_module)

0.8907103825136612
0.10655737704918032
5856


Sanity Checking: 0it [00:00, ?it/s]

Confusion matrix
tensor([[12,  9],
        [35,  8]], device='cuda:0')


Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

Confusion matrix
tensor([[137,   5],
        [  6, 390]], device='cuda:0')


Validation: 0it [00:00, ?it/s]

Confusion matrix
tensor([[138,   4],
        [  6, 390]], device='cuda:0')


Validation: 0it [00:00, ?it/s]

Confusion matrix
tensor([[139,   3],
        [  6, 390]], device='cuda:0')


In [ ]:
torch.save(model.state_dict(), 'cnn_pneumonia-11.pth')

In [ ]:
print(summary(model.to("cuda"),input_size=(32,3,224,224)))

In [ ]:
trainer.test(model, data_module)
print(model.confusion_matrix.compute())

0.8907103825136612
0.10655737704918032
5856


Testing: 0it [00:00, ?it/s]

/usr/local/lib/python3.7/dist-packages/torchmetrics/utilities/prints.py:36: UserWarning: No negative samples in targets, false positive value should be meaningless. Returning zero tensor in false positive score
  warnings.warn(*args, **kwargs)
/usr/local/lib/python3.7/dist-packages/torchmetrics/utilities/prints.py:36: UserWarning: No positive samples in targets, true positive value should be meaningless. Returning zero tensor in true positive score
  warnings.warn(*args, **kwargs)


────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        test_acc            0.8333333134651184
       test_auroc           0.9453046321868896
         test_f1            0.7989665865898132
        test_loss            0.746821939945221
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
tensor([[131, 103],
        [  1, 389]])


In [ ]:
%reload_ext tensorboard
%tensorboard --logdir lightning_logs/

In [ ]:
import seaborn as sns
fig = plt.figure(figsize=(5,5))
sns.heatmap(model.confusion_matrix.compute(), annot=True,cmap='summer',square=True, xticklabels=['normal','pneumonia'],yticklabels=['normal','pneumonia'],fmt="d", cbar=False,linewidths=3, linecolor='b')
plt.title('Confusion matrix - DenseNet',fontsize=15)